In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Unrecognized magic `%%shell`.

Julia does not use the IPython `%magic` syntax.   To interact with the IJulia kernel, use `IJulia.somefunction(...)`, for example.  Julia macros, string macros, and functions can be used to accomplish most of the other functionalities of IPython magics.


In [ ]:
versioninfo()

Julia Version 1.8.2
Commit 36034abf260 (2022-09-29 15:21 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, broadwell)
  Threads: 2 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = 2




<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />

In [ ]:
using Pkg

In [ ]:
Pkg.add(url="https://github.com/JuliaQuantumControl/GRAPELinesearchAnalysis.jl.git")

In [ ]:
Pkg.add("QuantumControl")

In [ ]:
using GRAPELinesearchAnalysis

In [ ]:
using QuantumControl

In [ ]:
Pkg.add("QuantumControlBase")

In [ ]:
using QuantumControlBase

In [ ]:
ϵ(t) = 0.2 * QuantumControl.Shapes.flattop(t, T=5, t_rise=0.3, func=:blackman);

In [ ]:
"""Two-level-system Hamiltonian."""
function hamiltonian(Ω=1.0, ϵ=ϵ)
    σ̂_z = ComplexF64[
        1  0
        0 -1
    ]
    σ̂_x = ComplexF64[
        0  1
        1  0
    ]
    Ĥ₀ = -0.5 * Ω * σ̂_z
    Ĥ₁ = σ̂_x
    return (Ĥ₀, (Ĥ₁, ϵ))
end;

H = hamiltonian();

In [ ]:
tlist = collect(range(0, 5, length=500));

In [ ]:
using LinearAlgebra

In [ ]:
function ket_(label)
    result = Dict("0" => Vector{ComplexF64}([1, 0]), "1" => Vector{ComplexF64}([0, 1]))
    return result[string(label)]
end;

In [ ]:
basis = [ket_("0"),ket_("1")];

In [ ]:
basis_tgt=Array[]
for i in 1:100
   b= transpose(RandU[i]) * basis
   push!(basis_tgt, b)

end


In [ ]:
objectives=Array[]
for i in 1:100
   obj = [Objective(initial_state = Ψ, target_state = Ψtgt, generator = H) for
    (Ψ, Ψtgt) ∈ zip(basis, basis_tgt[i])];
    push!(objectives, obj)
end




In [ ]:
problem=[]
for i in 1:100

    prob = ControlProblem(
    objectives=objectives[i],
    tlist=tlist,
    pulse_options=Dict(),
    iter_stop=500,
    J_T=QuantumControl.Functionals.J_T_sm,
    check_convergence=res -> begin
        ((res.J_T < 1e-3) && (res.converged = true) && (res.message = "J_T < 10⁻³"))
    end,
    );
    push!(problem, prob)
end

In [ ]:
Pkg.add("QuantumPropagators")
using QuantumPropagators

   Resolving package versions...
    Updating `~/.julia/environments/v1.8/Project.toml`
  [7bf12567] + QuantumPropagators v0.5.0
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [ ]:
function store_pulses(wrk, iteration, _...)
    L = length(wrk.controls)
    ϵ_opt = reshape(wrk.pulsevals, L, :)
    opt_pulses = [QuantumControl.Controls.discretize_on_midpoints(ϵ_opt[l, :], tlist) for l=1:L]
    return Tuple(opt_pulses)
end

store_pulses (generic function with 1 method)

In [ ]:
Pkg.add("GRAPE")
using GRAPE

   Resolving package versions...
    Updating `~/.julia/environments/v1.8/Project.toml`
  [6b52fcaf] + GRAPE v0.5.4
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [ ]:
wrk=[]
for i in 1:100
  wrkk=GRAPE.GrapeWrk(problem[i]; )
  push!(wrk, wrkk)
end


In [ ]:
results=[]

for i in 1:100
  opt_result_LBFGSB = optimize(

      problem[i],
      method = :grape,
      force = true,
      info_hook = QuantumControlBase.chain_infohooks(

          QuantumControl.GRAPE.print_table,
          store_pulses
      )
  );
  push!(results, opt_result_LBFGSB)
end

#opt_result_LBFGSB

 iter.        J_T     |∇J_T|       ΔJ_T   FG(F)    secs
     0   7.03e-01   8.41e-02        n/a    1(0)     0.0
     1   6.96e-01   8.38e-02  -7.05e-03    1(0)     0.0
     2   4.18e-01   6.67e-02  -2.78e-01    3(0)     0.1
     3   2.99e-01   1.25e-01  -1.19e-01    1(0)     0.0
     4   1.32e-01   9.47e-02  -1.67e-01    1(0)     0.0
     5   1.00e-01   7.99e-02  -3.21e-02    1(0)     0.0
     6   8.07e-04   3.19e-03  -9.93e-02    1(0)     0.0
 iter.        J_T     |∇J_T|       ΔJ_T   FG(F)    secs
     0   5.44e-01   1.20e-01        n/a    1(0)     0.1
     1   5.29e-01   1.20e-01  -1.45e-02    1(0)     0.0
     2   2.49e-01   9.68e-02  -2.80e-01    3(0)     0.1
     3   4.26e-02   4.09e-02  -2.06e-01    2(0)     0.0
     4   6.01e-03   2.30e-02  -3.65e-02    1(0)     0.0
     5   1.52e-04   3.41e-03  -5.86e-03    1(0)     0.0
 iter.        J_T     |∇J_T|       ΔJ_T   FG(F)    secs
     0   9.81e-01   3.87e-02        n/a    1(0)     0.0
     1   9.79e-01   4.02e-02  -1.53e-03    1(0) 

In [ ]:
J_F_t_const_1G=[]
for i in 1:100
  J_F=results[i].J_T
  push!(J_F_t_const_1G, J_F)
end


In [ ]:
N=100
J_F_avg_const_1G=0
sum=0
for i in 1:N

  sum=J_F_const_1G[i][end] + sum

end
J_F_avg_const_1G= sum/N

J_F_avg_const_1G

0.00029999915999999986

In [ ]:
t_conv_const_1G=[]

for i in 1:100
  t_sec = 0
  t_conv_i = []
   for j = 1: length(iter_sec_const_1G[i])
      t_sec = iter_sec_const_1G[i][j] + t_sec
      push!(t_conv_i, t_sec)
  end
  push!(t_conv_const_1G, t_conv_i)
end
println(t_conv_const_1G)

Any[Any[0.0, 0.0, 0.1, 0.1, 0.1, 0.1, 0.1], Any[0.1, 0.1, 0.2, 0.2, 0.2, 0.2], Any[0.0, 0.0, 0.1, 0.2, 0.2, 0.2, 0.2, 0.2], Any[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Any[0.0, 0.0, 0.1, 0.1, 0.1], Any[0.0, 0.0, 0.1, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2], Any[0.0, 0.0, 0.1, 0.1, 0.1], Any[0.0, 0.1, 0.2, 0.30000000000000004, 0.30000000000000004, 0.30000000000000004, 0.30000000000000004, 0.30000000000000004, 0.30000000000000004], Any[0.0, 0.1, 0.2, 0.2, 0.2], Any[0.0, 0.0, 0.1, 0.2, 0.2, 0.2, 0.2, 0.2], Any[0.0, 0.0, 0.1, 0.1, 0.1, 0.1], Any[0.0, 0.0, 0.1, 0.1, 0.2, 0.2, 0.2, 0.30000000000000004, 0.30000000000000004], Any[0.0, 0.0, 0.1, 0.1, 0.1, 0.1], Any[0.0, 0.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1], Any[0.0, 0.0, 0.1, 0.2, 0.2, 0.2, 0.2, 0.2], Any[0.0, 0.0, 0.1, 0.1, 0.1], Any[0.0, 0.0, 0.1, 0.1, 0.2], Any[0.0, 0.0, 0.1, 0.2, 0.2, 0.2, 0.2, 0.2], Any[0.0, 0.1, 0.2, 0.2, 0.2, 0.2], Any[0.0, 0.0, 0.1, 0.2, 0.2, 0.2, 0.2, 0.2], Any[0.0, 0.0, 0.1, 0.2, 0.30000000000000004, 0.4, 0.4, 0.5, 0.5], An

In [ ]:
N=100
t_conv_avg_const_1G=0
sum=0
for i in 1:N

  sum=t_conv_const_1G[i][end] + sum

end

t_conv_avg_const_1G= sum/N
println(t_conv_avg_const_1G)

0.19899999999999995


In [ ]:
N=100
t_it_avg_1G=0
sum=0
for i in 1:N
  for j in 1:length(iter_sec_100_1G[i])
  #sum=iter_sec_100_1G[i][j] + sum
  t_it_avg_1G= t_conv_100_1G[1][end]/length(iter_sec_100_1G[i])
  end
end
println(t_it_avg_1G)